In [ ]:
# setup vars/funcs for scanCore

def dprint(text, end="\r"):
    if DEBUG:
        print(text, end=end)

DEBUG = True

In [ ]:
#   creates ip ranges

import random

# create a list of ipv4 ranges
ip_lists = []
for i in range(255):
    for j in range(255):
        ip_lists.append(f"{i}.{j}.0.0/16")
random.shuffle(ip_lists)

# ip_lists = ["172.64.0.0/16","172.65.0.0/16","172.66.0.0/16"] # remove for final vers

In [ ]:
#     checks if the ip is runnging a server

from itertools import count
import json
import os
import time

upHosts = []
results = []
start = time.time()

def scan(ip_list):
    try:
        import masscan
        import json

        scanner = masscan.PortScanner()
        scanner.scan(ip_list, ports='25565', arguments='--max-rate 100')
        res = json.loads(scanner.scan_result)
        global results
        results.append(list(res["scan"].keys()))
    except Exception as e:
        print(e)

import threading

print("Starting threads")
threads = []
for host in ip_lists:
    maxActive = 10

    while True: # wait for a thread to finish
        c = 0
        for i in threads:
            if i.is_alive():
                c += 1
        if c <= maxActive:
            break
        
        time.sleep(0.1)
            
    
    t = threading.Thread(target=scan, args=(host,))
    t.start()
    threads.append(t)

    c = 0
    for result in results:
        c += len(result)
    print(f"Started thread for {host} | Total Threads: {len(threads)} | Total Found {c}")

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break
c = 0
for result in results:
    c += len(result)

print(f"found {c} hosts | average time/list {round(((time.time()-start)/(len(ip_lists))),2)} {' '*40}")

In [ ]:
#     checks if the ip is runnging a minecraft server

import time

start_time = time.time()
res = []
lst = []
for result in results:
    for host in result:
        lst.append(host)

def check(host):
    try:
        import mcstatus
        print("\r",end="")
        server = mcstatus.JavaServer.lookup(host)
        status = server.status()
        description = status.description
        #remove color codes and whitespace
        description = description.replace("§0","").replace("§1","").replace("§2","").replace("§3","").replace("§4","").replace("§5","").replace("§6","").replace("§7","").replace("§8","").replace("§9","").replace("§a","").replace("§b","").replace("§c","").replace("§d","").replace("§e","").replace("§f","").replace("§k","").replace("§l","").replace("§m","").replace("§n","").replace("§o","").replace("§r","").replace(" ","")
        dprint(f"{host} ({description}): has {status.players.online} players and replied in {round(status.latency,2)} ms\n")

        res.append(host)
    except Exception as e:
        pass

import threading

print("Starting threads")
threads = []
for host in lst:
    thread = threading.Thread(target=check, args=(host,))
    thread.start()
    threads.append(thread)

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break


In [ ]:
#   Deal with the data
print(f"found {len(res)} servers")